In [1]:
import os
import sys
import cv2
import math
import numpy as np
from lib import quantification
from lib import help_functions
from scipy.ndimage.morphology import distance_transform_edt
import scipy.ndimage as ndi
from tqdm import tqdm
from matplotlib import pyplot as plt

import configparser


sys.path.insert(0, './lib/')


from skimage.data import binary_blobs
from skimage.morphology import medial_axis, skeletonize, skeletonize_3d,thin
from skimage import data
from skimage.util import invert
from skimage.morphology import remove_small_objects
from skimage import morphology
from skimage import img_as_float
from skimage import img_as_ubyte
from skimage import img_as_uint
from skimage import io
from skimage import util
from scipy.ndimage.filters import convolve
from scipy import signal
from skimage.filters import threshold_otsu
from skimage.filters import rank
from skimage import img_as_float #convert img, we should use this function. Do Not Use np.astype

#np.set_printoptions(threshold=np.nan)


In [2]:
def load_vessel_ori_map(ORI_PATH, shape_imgs,num_img = 10 ,resize_factor = 0.5):
    img_h, img_w = shape_imgs[0], shape_imgs[1]
    img_h = int(img_h * resize_factor); img_w = int(img_w * resize_factor)
    ori_vessel_imgs = np.zeros((num_img, img_h, img_w,3),dtype=np.uint8)    
    idx =0 
    
    for count, filename in enumerate(sorted(os.listdir(ORI_PATH)), start=0):
        if filename.startswith(".ipynb") == False:
            temp = cv2.imread(ORI_PATH + filename)
            print('ori vessel imgs shape : ', np.shape(temp))
            temp = cv2.resize(temp, (img_w, img_h), interpolation = cv2.INTER_CUBIC)

            ori_vessel_imgs[idx] = temp.astype(np.uint8)
            idx = idx+1
 
            
    return ori_vessel_imgs

def load_vessel_ground_truth(GT_PATH, shape_imgs,num_img=10,resize_factor = 0.5):
    img_h, img_w = shape_imgs[0], shape_imgs[1]
    img_h = int(img_h * resize_factor); img_w = int(img_w * resize_factor)
    gt_vessel_imgs = np.zeros((num_img, img_h, img_w),dtype=np.uint8)    
    print('gt vessel imgs shape : ',np.shape(gt_vessel_imgs))
    idx =0 
    
    for count, filename in enumerate(sorted(os.listdir(GT_PATH)), start=0):
        if filename.startswith(".ipynb") == False:
            temp = (cv2.imread(GT_PATH + filename))[:,:,0]
            print('ori vessel imgs shape : ', np.shape(temp))
            temp = cv2.resize(temp, (img_w, img_h), interpolation = cv2.INTER_CUBIC)
            
            ret,thresh = cv2.threshold(temp,100,255,cv2.THRESH_BINARY)
            thresh = np.array(thresh,np.uint8)
            gt_vessel_imgs[idx] = thresh
            
            idx = idx+1

            
    return gt_vessel_imgs


def get_shape_of_imgs(DATA_PATH):
    for count, filename in enumerate(sorted(os.listdir(DATA_PATH)), start=0):
        if filename.startswith(".ipynb") == False:
            temp_imgs = cv2.imread(DATA_PATH + filename)
            return np.shape(temp_imgs)


In [3]:
data_path = './data/conj_train/acc_valid/'
gt_path = './data/conj_train/acc_gt/'

SHAPE_IMG = get_shape_of_imgs(data_path)
ORI_DATA = load_vessel_ori_map(data_path, SHAPE_IMG)
GT_DATA = load_vessel_ground_truth(gt_path, SHAPE_IMG)

print(np.shape(ORI_DATA), np.shape(GT_DATA))

ori vessel imgs shape :  (1800, 2200, 3)
ori vessel imgs shape :  (1800, 2200, 3)
ori vessel imgs shape :  (1800, 2200, 3)
ori vessel imgs shape :  (1800, 2200, 3)
ori vessel imgs shape :  (1800, 2200, 3)
ori vessel imgs shape :  (1800, 2200, 3)
ori vessel imgs shape :  (1800, 2200, 3)
ori vessel imgs shape :  (1800, 2200, 3)
ori vessel imgs shape :  (1800, 2200, 3)
ori vessel imgs shape :  (1800, 2200, 3)
gt vessel imgs shape :  (10, 900, 1100)
ori vessel imgs shape :  (1800, 2200)
ori vessel imgs shape :  (1800, 2200)
ori vessel imgs shape :  (1800, 2200)
ori vessel imgs shape :  (1800, 2200)
ori vessel imgs shape :  (1800, 2200)
ori vessel imgs shape :  (1800, 2200)
ori vessel imgs shape :  (1800, 2200)
ori vessel imgs shape :  (1800, 2200)
ori vessel imgs shape :  (1800, 2200)
ori vessel imgs shape :  (1800, 2200)
(10, 900, 1100, 3) (10, 900, 1100)
